# Análise de Emoções nos Hinos da ICM
Este notebook utiliza modelos de NLP para identificar emoções predominantes nos textos dos hinos da coletânea principal da Igreja Cristã Maranata.

O objetivo é classificar os hinos quanto às emoções transmitidas, visualizar a distribuição das emoções e salvar os resultados para futuras análises.

---
**Conteúdo do notebook:**
- Carregamento do modelo de emoções (BERT-pt-emotion)
- Processamento dos textos dos hinos para análise de emoções
- Visualização da distribuição das emoções dominantes
- Estatísticas e exemplos por emoção
- Visualização detalhada (heatmap)
- Extração das 3 emoções mais fortes por hino
- Salvamento dos resultados enriquecidos

Este material é público e pode ser compartilhado para fins de pesquisa, estudo ou divulgação cultural.

# Parte 7 - Análise de Emoção

---

**A seguir:** Carregamento do modelo de emoções e dos dados dos hinos, definição das funções para processamento dos textos e classificação das emoções.

In [ ]:
from transformers import pipeline
import pandas as pd
import torch

hinos_analise: pd.DataFrame = pd.read_pickle(
    "..\\assets\\hinos_analise_embeddings_complete.pkl"
)

# Carregar modelo multilíngue de emoções
model_name = "pysentimiento/bert-pt-emotion"
classifier = pipeline(
    "text-classification",
    model=model_name,
    tokenizer=model_name,
    top_k=None,  # Substitui return_all_scores=True (depreciado)
    max_length=512,  # Limite máximo de tokens do BERT
    truncation=True,  # Trunca textos longos automaticamente
    device=-1,  # Força uso da CPU para evitar problemas de memória
)


# Função para juntar e truncar tokens já tratados
def texto_truncado_tokens(tokens, max_tokens=400):
    if not isinstance(tokens, list):
        return ""
    # Trunca a lista de tokens e junta em uma string
    return " ".join(tokens[:max_tokens])


# Classificar emoções usando a coluna token_no_stops
def analisar_emocoes_tokens(tokens):
    try:
        texto = texto_truncado_tokens(tokens)
        if not texto.strip():
            return {}
        resultado = classifier(texto)
        if isinstance(resultado, list) and len(resultado) > 0:
            emocoes_lista = resultado[0]
            if isinstance(emocoes_lista, list):
                return {r["label"]: r["score"] for r in emocoes_lista}
        return {}
    except Exception as e:
        print(f"Erro ao processar texto: {str(e)[:100]}...")
        return {}


# Verificar alguns textos antes de processar tudo
print("Verificando tamanhos dos textos...")
print(f"Total de hinos: {len(hinos_analise)}")

---

**A seguir:** Processamento dos hinos em lotes para análise de emoções, salvando os resultados e exibindo amostras dos dados processados.

In [ ]:
# Processar todos os hinos (pode demorar alguns minutos)
print("Processando análise de emoções para todos os hinos...")
print("Isso pode levar alguns minutos. Progresso:")

import time

start_time = time.time()

# Processar em lotes para mostrar progresso
batch_size = 50
total_batches = len(hinos_analise) // batch_size + 1

all_emotions = []
for i in range(0, len(hinos_analise), batch_size):
    batch = hinos_analise.iloc[i : i + batch_size]
    batch_emotions = batch["tokens_no_stops"].apply(analisar_emocoes_tokens)
    all_emotions.extend(batch_emotions.tolist())

    current_batch = i // batch_size + 1
    print(
        f"Lote {current_batch}/{total_batches} concluído ({i+len(batch)}/{len(hinos_analise)} hinos)"
    )

# Adicionar resultados ao dataframe
hinos_analise["emocoes"] = all_emotions

end_time = time.time()
print(f"\nProcessamento concluído em {end_time - start_time:.1f} segundos!")
print(f"Total de hinos processados: {len(hinos_analise)}")

# Mostrar amostra dos resultados
print("\nAmostra dos resultados:")
for idx, row in hinos_analise.head(10).iterrows():
    emocoes = row["emocoes"]
    if emocoes:
        top_emocao = max(emocoes.items(), key=lambda x: x[1])
        print(f"{row['nome'][:40]}... -> {top_emocao[0]}: {top_emocao[1]:.3f}")
    else:
        print(f"{row['nome'][:40]}... -> Erro no processamento")

---

**A seguir:** Visualização e análise dos resultados das emoções, incluindo distribuição das emoções dominantes, estatísticas e exemplos por emoção.

In [ ]:
# Análise e visualização dos resultados de emoções
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Verificar se os dados foram processados
if "emocoes" not in hinos_analise.columns:
    print("Execute a célula anterior primeiro para processar as emoções!")
else:
    print("Analisando resultados das emoções...")

    # Extrair a emoção dominante de cada hino
    emocoes_dominantes = []
    scores_dominantes = []

    for emocoes in hinos_analise["emocoes"]:
        if emocoes:
            top_emocao = max(emocoes.items(), key=lambda x: x[1])
            emocoes_dominantes.append(top_emocao[0])
            scores_dominantes.append(top_emocao[1])
        else:
            emocoes_dominantes.append("unknown")
            scores_dominantes.append(0.0)

    hinos_analise["emocao_dominante"] = emocoes_dominantes
    hinos_analise["score_dominante"] = scores_dominantes

    # 1. Distribuição das emoções dominantes
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    emocao_counts = pd.Series(emocoes_dominantes).value_counts().head(10)
    emocao_counts.plot(kind="bar", color="skyblue")
    plt.title("Top 10 Emoções Dominantes nos Hinos")
    plt.xlabel("Emoção")
    plt.ylabel("Quantidade de Hinos")
    plt.xticks(rotation=45)

    # 2. Distribuição dos scores das emoções dominantes
    plt.subplot(1, 2, 2)
    plt.hist(scores_dominantes, bins=20, color="lightcoral", alpha=0.7)
    plt.title("Distribuição dos Scores das Emoções Dominantes")
    plt.xlabel("Score da Emoção")
    plt.ylabel("Frequência")

    plt.tight_layout()
    plt.show()

    # 3. Estatísticas das emoções
    print("\n=== ESTATÍSTICAS DAS EMOÇÕES ===")
    print(f"Total de hinos analisados: {len(hinos_analise)}")
    print(f"Hinos com erro no processamento: {emocoes_dominantes.count('unknown')}")
    print(
        f"Score médio das emoções dominantes: {np.mean([s for s in scores_dominantes if s > 0]):.3f}"
    )

    print("\nTop 10 emoções mais frequentes:")
    for i, (emocao, count) in enumerate(emocao_counts.head(10).items(), 1):
        percentage = (count / len(hinos_analise)) * 100
        print(f"{i:2d}. {emocao:15s}: {count:3d} hinos ({percentage:4.1f}%)")

    # 4. Exemplos de hinos por emoção dominante
    print("\n=== EXEMPLOS POR EMOÇÃO ===")
    for emocao in emocao_counts.head(5).index:
        hinos_emocao = hinos_analise[hinos_analise["emocao_dominante"] == emocao]
        exemplo = hinos_emocao.iloc[0]
        print(f"\n{emocao.upper()} (score: {exemplo['score_dominante']:.3f}):")
        print(f"  Hino: {exemplo['nome']}")
        print(f"  Texto: {exemplo['texto_limpo'][:100]}...")

    print(f"\nProcessamento concluído! Dados salvos com análise de emoções.")

---

**A seguir:** Análise das emoções ignorando a categoria 'neutral', visualização da distribuição e exemplos por emoção sem neutralidade.

In [ ]:
# Análise ignorando a emoção 'neutral'
print("Analisando emoções ignorando 'neutral'...")
emocoes_dominantes_sem_neutral = []
scores_dominantes_sem_neutral = []
for emocoes in hinos_analise["emocoes"]:
    if emocoes:
        # Remove 'neutral' se existir
        emocoes_filtrado = {k: v for k, v in emocoes.items() if k != "neutral"}
        if emocoes_filtrado:
            top_emocao = max(emocoes_filtrado.items(), key=lambda x: x[1])
            emocoes_dominantes_sem_neutral.append(top_emocao[0])
            scores_dominantes_sem_neutral.append(top_emocao[1])
        else:
            emocoes_dominantes_sem_neutral.append("unknown")
            scores_dominantes_sem_neutral.append(0.0)
    else:
        emocoes_dominantes_sem_neutral.append("unknown")
        scores_dominantes_sem_neutral.append(0.0)

hinos_analise["emocao_dominante_sem_neutral"] = emocoes_dominantes_sem_neutral
hinos_analise["score_dominante_sem_neutral"] = scores_dominantes_sem_neutral

# Visualização e estatísticas
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
emocao_counts = pd.Series(emocoes_dominantes_sem_neutral).value_counts().head(10)
emocao_counts.plot(kind="bar", color="orange")
plt.title("Top 10 Emoções Dominantes (sem neutral)")
plt.xlabel("Emoção")
plt.ylabel("Quantidade de Hinos")
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.hist(
    [s for s in scores_dominantes_sem_neutral if s > 0],
    bins=20,
    color="purple",
    alpha=0.7,
)
plt.title("Distribuição dos Scores das Emoções Dominantes (sem neutral)")
plt.xlabel("Score da Emoção")
plt.ylabel("Frequência")

plt.tight_layout()
plt.show()

print("\n=== ESTATÍSTICAS DAS EMOÇÕES (sem neutral) ===")
print(f"Total de hinos analisados: {len(hinos_analise)}")
print(
    f"Hinos com erro no processamento: {emocoes_dominantes_sem_neutral.count('unknown')}"
)
print(
    f"Score médio das emoções dominantes: {np.mean([s for s in scores_dominantes_sem_neutral if s > 0]):.3f}"
)

print("\nTop 10 emoções mais frequentes (sem neutral):")
for i, (emocao, count) in enumerate(emocao_counts.head(10).items(), 1):
    percentage = (count / len(hinos_analise)) * 100
    print(f"{i:2d}. {emocao:15s}: {count:3d} hinos ({percentage:4.1f}%)")

print("\n=== EXEMPLOS POR EMOÇÃO (sem neutral) ===")
for emocao in emocao_counts.head(5).index:
    hinos_emocao = hinos_analise[
        hinos_analise["emocao_dominante_sem_neutral"] == emocao
    ]
    exemplo = hinos_emocao.iloc[0]
    print(f"\n{emocao.upper()} (score: {exemplo['score_dominante_sem_neutral']:.3f}):")
    print(f"  Hino: {exemplo['nome']}")
    print(f"  Texto: {exemplo['texto_limpo'][:100]}...")

---

**A seguir:** Visualização detalhada dos scores de emoções por hino, utilizando heatmap para destacar as emoções principais em uma amostra dos hinos.

In [ ]:
# Visualização detalhada: Heatmap de emoções por hino
import seaborn as sns
import matplotlib.pyplot as plt

# Verificar se os dados foram processados
if "emocoes" not in hinos_analise.columns:
    print("Execute as células anteriores primeiro para processar as emoções!")
else:
    print("Criando heatmap de emoções...")

    # Expandir as emoções em colunas numéricas para uma amostra
    # (usar todos os hinos seria muito denso para visualizar)
    amostra_viz = hinos_analise.head(20).copy()  # Primeiros 20 hinos

    # Criar dataframe com as emoções expandidas
    emocoes_expandidas = []
    nomes_hinos = []

    for idx, row in amostra_viz.iterrows():
        if row["emocoes"]:
            emocoes_expandidas.append(row["emocoes"])
            nomes_hinos.append(row["nome"][:30] + "...")  # Truncar nomes longos

    if emocoes_expandidas:
        emo_df = pd.DataFrame(emocoes_expandidas, index=nomes_hinos)

        # Selecionar apenas as emoções mais comuns para visualização
        emocoes_principais = emo_df.mean().nlargest(10).index
        emo_df_filtrado = emo_df[emocoes_principais]

        plt.figure(figsize=(12, 8))
        sns.heatmap(
            emo_df_filtrado,
            annot=True,
            cmap="YlOrRd",
            fmt=".2f",
            cbar_kws={"label": "Score da Emoção"},
        )
        plt.title("Distribuição de Emoções por Hino\n(Amostra dos primeiros 20 hinos)")
        plt.xlabel("Emoção")
        plt.ylabel("Hino")
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()

        print(
            f"Heatmap criado com {len(emo_df_filtrado)} hinos e {len(emocoes_principais)} emoções principais."
        )
        print(f"Emoções visualizadas: {list(emocoes_principais)}")
    else:
        print("Nenhuma emoção foi processada com sucesso. Verifique os dados.")

---

**A seguir:** Extração das 3 emoções mais fortes por hino, para facilitar análises e visualizações futuras.

In [ ]:
# Criar coluna com as 3 emoções mais dominantes (sem neutral)
def top3_emocoes_sem_neutral(emocoes):
    if not emocoes:
        return []
    # Remove 'neutral'
    emocoes_filtrado = {k: v for k, v in emocoes.items() if k != "neutral"}
    # Ordena por score (do maior para o menor)
    top3 = sorted(emocoes_filtrado.items(), key=lambda x: x[1], reverse=True)[:3]
    # Retorna só os nomes das emoções
    return [e[0] for e in top3]


hinos_analise["top3_emocoes"] = hinos_analise["emocoes"].apply(top3_emocoes_sem_neutral)

# Exemplo de visualização
print(hinos_analise[["nome", "top3_emocoes"]].head(10))

---

**A seguir:** Salvamento dos resultados enriquecidos com análise de emoções para uso em outras etapas ou aplicações.

In [ ]:
hinos_analise_save = hinos_analise.copy()
hinos_analise_save.drop(
    columns=[
        "emocao_dominante",
        "score_dominante",
        "emocao_dominante_sem_neutral",
        "score_dominante_sem_neutral",
    ],
    inplace=True,
)
hinos_analise_save.to_pickle("..\\assets\\hinos_analise_com_emocoes.pkl")

---

**Fim do notebook:** Finalização do processamento, com os dados prontos para exportação e uso em outras análises ou aplicações.